# EP - Classificador de Grão de Arroz


##Parte 1 (4 pontos): Alinhar os grãos horizontalmente - “alinha.py”

In [ ]:
# Bibliotecas utilizadas na parte 1
import cv2
import numpy as np
import os
from time import time

In [ ]:
# Retorna o centro de massa do grão e sua orientação em rad
def encontrarCentroOrientacao(bin):
  contornos, _ = cv2.findContours(bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  contorno_maior = max(contornos, key=cv2.contourArea)
  M = cv2.moments(contorno_maior)
  cen_x = int(M["m10"] / M["m00"])
  cen_y = int(M["m01"] / M["m00"])
  theta = 0.5*np.arctan2(2*M["mu11"], M["mu20"]-M["mu02"])
  return [cen_x, cen_y, theta]

In [ ]:
# Salva a imagem intermediária figurab para teste
def figura_2b(data, original, name):
  a = original.copy()
  inicial = [int(data[0]-70*np.cos(data[2])), int(data[1]-70*np.sin(data[2]))]
  final = [int(data[0]+70*np.cos(data[2])), int(data[1]+70*np.sin(data[2]))]
  cv2.line(a, inicial, final, [0, 0, 255], 1) # Desenha uma linha indicando a orientação
  for i in range(-2,2,1):
    for j in range(-2,2,1):
      a[data[1]+i,data[0]+j] = [255,0,0]      # Faz um quadrado no centro de massa
  cv2.imwrite(name[:-4]+"-figurab.jpg", a)    # Salva com o nome da figura + (figurab.jpg)

In [ ]:
# Retorna uma lista com todos os arquivos presentes no diretório com a extensão fornecida
def listar_Extensao(diretorio_Base, extensao):
    if (not os.path.exists(diretorio_Base)):                  # Caso não exista o diretório
        print("\nDiretório inexistente\n")
        return []

    os.chdir(diretorio_Base)
    file_path = []
    for file in os.listdir():                                 # Lista todos os arquivos do diretório
        if file.endswith("." + extensao):
            file_path.append(f"{diretorio_Base}/{file}")

    if len(file_path) == 0:                                   # Caso não exista arquivos
        return []

    return file_path

In [ ]:
def alinha(diretorio):
  t0=time()
  arquivos = listar_Extensao(diretorio, 'jpg') # Recebe o nome de todas as imagens

  for name in arquivos:
    original = cv2.imread(name, 1)
    img = cv2.imread(name, 0)
    rows, cols = original.shape[:2]

    # Binarizar a imagem
    lim, bin = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    #Encontrar o centro e a orientação
    data = encontrarCentroOrientacao(bin)
    #Salvar figuras para teste
    #figura_2b(data, original, name)

    #Rotacionar e centralizar
    rotacao = cv2.getRotationMatrix2D([data[0],data[1]], np.degrees(data[2]), 1.0) # Rotacionar
    rotacao[0][2] = rotacao[0][2]-(data[0]-rows/2)
    rotacao[1][2] = rotacao[1][2]-(data[1]-cols/2)
    img_alinhada = cv2.warpAffine(original, rotacao, (cols,rows)) # Centralizar

    cv2.imwrite(name[:-3]+"png", img_alinhada)

  print('Alinha.py:', len(arquivos), 'arquivos processados. Tempo de processamento:', round(time()-t0, 3), 's.')

In [ ]:
alinha('/content/drive/MyDrive/rice_1to50/orientacao/')

Alinha.py: 6 arquivos processados. Tempo de processamento: 0.059 s.


In [ ]:
alinha('/content/drive/MyDrive/rice_1to50/Arborio/')
alinha('/content/drive/MyDrive/rice_1to50/Basmati/')
alinha('/content/drive/MyDrive/rice_1to50/Ipsala/')
alinha('/content/drive/MyDrive/rice_1to50/Jasmine/')
alinha('/content/drive/MyDrive/rice_1to50/Karacadag/')

Alinha.py: 50 arquivos processados. Tempo de processamento: 0.504 s.
Alinha.py: 50 arquivos processados. Tempo de processamento: 0.516 s.
Alinha.py: 50 arquivos processados. Tempo de processamento: 0.547 s.
Alinha.py: 50 arquivos processados. Tempo de processamento: 0.55 s.
Alinha.py: 50 arquivos processados. Tempo de processamento: 0.534 s.


##Parte 2 (6 pontos): Classificar as imagens não-alinhadas e alinhadas - “class_orig.py e class_alin.py”

### Classificar as imagens não-alinhadas

In [ ]:
#Bibliotecas utilizadas
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras import optimizers
import numpy as np; import sys; import os; from time import time
from tensorflow.keras.utils import plot_model

In [ ]:
# Criar valores AX AY e QX QY
def fiveFoldCrossValidation(inicio, fim, diretorio, extensao):
  #Gera a lista com todas as imagens para treinamento e teste
  arquivos = listar_Extensao(diretorio+'Arborio/', extensao)         # indice 0
  arquivos.extend(listar_Extensao(diretorio+'Basmati/', extensao))   # indice 1
  arquivos.extend(listar_Extensao(diretorio+'Ipsala/', extensao))    # indice 2
  arquivos.extend(listar_Extensao(diretorio+'Jasmine/', extensao))   # indice 3
  arquivos.extend(listar_Extensao(diretorio+'Karacadag/', extensao)) # indice 4

  first_train = 1
  first_test = 1
  for i in range(0, len(arquivos)):
    if i%50 >= inicio and i%50 <= fim:  #Valores entre inicio e fim - TESTE
      if first_test == 1:
        QX = np.array(cv2.imread(arquivos[i], 0))
        QX = QX.reshape(1, QX.shape[0], QX.shape[1])
        QY = np.array(int(i/50)) # Pega indice da imagem (0 a 4)
        first_test = 0
      else:
        QX2 = np.array(cv2.imread(arquivos[i], 0))
        QX2 = QX2.reshape(1, QX2.shape[0], QX2.shape[1])
        QX = np.concatenate([QX, QX2], 0)
        QY = np.append(QY, int(i/50))

    else: #Valores fora do inicio e fim - TREINAMENTO
      if first_train == 1:
        AX = np.array(cv2.imread(arquivos[i], 0))
        AX = AX.reshape(1, AX.shape[0], AX.shape[1])
        AY = np.array(int(i/50))
        first_train = 0
      else:
        AX2 = np.array(cv2.imread(arquivos[i], 0))
        AX2 = AX2.reshape(1, AX2.shape[0], AX2.shape[1])
        AX = np.concatenate([AX, AX2], 0)
        AY = np.append(AY, int(i/50))

  print("Tamanho das matrizes: AX =", AX.shape, "AY =", AY.shape, "QX =", QX.shape, "QY =", QY.shape)
  return AX, AY, QX, QY

In [ ]:
def modeloConvolucional(AX, AY, QX, QY):
  AX = 255 - AX
  QX = 255 - QX
  nclasses = 10
  nl, nc = AX.shape[1], AX.shape[2]

  AX = (AX.astype('float32')/255.0)-0.5
  QX = (QX.astype('float32')/255.0)-0.5
  AX = np.expand_dims(AX,axis=3)
  QX = np.expand_dims(QX,axis=3)

  # Criando modelo, nomeando todos os parâmetros da rede
  model = Sequential()
  model.add(Conv2D(20, kernel_size=(5,5), activation='relu', input_shape=(nl, nc, 1)))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(40, kernel_size=(5,5), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(200, activation='relu'))
  model.add(Dense(nclasses, activation='softmax'))
  model.compile(optimizer=optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  #plot_model(model, to_file='cnn1.png', show_shapes=True);
  #model.summary()

  t0 = time()
  model.fit(AX, AY, batch_size=100, epochs=30, verbose=2)
  print("Tempo de treino: %.2f s"%(time()-t0))

  score = model.evaluate(QX, QY, verbose=False)
  print('Test loss: %.4f'%(score[0]))
  print('Test accuracy: %.2f %%'%(100*score[1]))
  print('Test error: %.2f %%'%(100*(1-score[1])))

  t2 = time()
  QP = np.argmax(model.predict(QX), 1)
  #print("Tempo de predicao: %.2f s"%(time()-t2))
  #nerro = np.count_nonzero(QP-QY)
  #print("nerro=%d"%(nerro))
  #model.save('cnn1.h5')

  return round(score[0],4), round(score[1],4)

In [ ]:
def class_orig(diretorio):
  AX, AY, QX, QY = fiveFoldCrossValidation(0, 9, diretorio, 'jpg')    #S1
  pS1, aS1 = modeloConvolucional(AX, AY, QX, QY)
  AX, AY, QX, QY = fiveFoldCrossValidation(10, 19, diretorio, 'jpg')  #S2
  pS2, aS2 = modeloConvolucional(AX, AY, QX, QY)
  AX, AY, QX, QY = fiveFoldCrossValidation(20, 29, diretorio, 'jpg')  #S3
  pS3, aS3 = modeloConvolucional(AX, AY, QX, QY)
  AX, AY, QX, QY = fiveFoldCrossValidation(30, 39, diretorio, 'jpg')  #S4
  pS4, aS4 = modeloConvolucional(AX, AY, QX, QY)
  AX, AY, QX, QY = fiveFoldCrossValidation(40, 49, diretorio, 'jpg')  #S5
  pS5, aS5 = modeloConvolucional(AX, AY, QX, QY)

  print('\n\nResultados class_orig: 5-fold cross validation:\n')
  print('Perda do Teste: ',pS1, pS2, pS3, pS4, pS5, 'Média:', round(np.mean([pS1, pS2, pS3, pS4, pS5]),4))
  print('Acurácia do Teste: ',aS1, aS2, aS3, aS4, aS5, 'Média:', round(np.mean([aS1, aS2, aS3, aS4, aS5]),4))
  print('Erro do Teste: ',round(1-aS1,4), round(1-aS2,4), round(1-aS3,4), round(1-aS4,4), round(1-aS5,4), 'Média:', round(np.mean([1-aS1, 1-aS2, 1-aS3, 1-aS4, 1-aS5]),4))

class_orig('/content/drive/MyDrive/rice_1to50/')

Tamanho das matrizes: AX = (200, 250, 250) AY = (200,) QX = (50, 250, 250) QY = (50,)
Epoch 1/30
2/2 - 2s - loss: 3.2982 - accuracy: 0.2900 - 2s/epoch - 1s/step
Epoch 2/30
2/2 - 0s - loss: 4.8271 - accuracy: 0.1650 - 229ms/epoch - 115ms/step
Epoch 3/30
2/2 - 0s - loss: 2.0030 - accuracy: 0.4000 - 212ms/epoch - 106ms/step
Epoch 4/30
2/2 - 0s - loss: 1.5263 - accuracy: 0.5350 - 214ms/epoch - 107ms/step
Epoch 5/30
2/2 - 0s - loss: 1.0838 - accuracy: 0.6700 - 211ms/epoch - 106ms/step
Epoch 6/30
2/2 - 0s - loss: 0.6528 - accuracy: 0.8650 - 208ms/epoch - 104ms/step
Epoch 7/30
2/2 - 0s - loss: 0.3682 - accuracy: 0.9600 - 214ms/epoch - 107ms/step
Epoch 8/30
2/2 - 0s - loss: 0.2743 - accuracy: 0.9450 - 214ms/epoch - 107ms/step
Epoch 9/30
2/2 - 0s - loss: 0.1937 - accuracy: 0.9450 - 210ms/epoch - 105ms/step
Epoch 10/30
2/2 - 0s - loss: 0.1361 - accuracy: 0.9600 - 210ms/epoch - 105ms/step
Epoch 11/30
2/2 - 0s - loss: 0.1077 - accuracy: 0.9700 - 212ms/epoch - 106ms/step
Epoch 12/30
2/2 - 0s - loss

In [ ]:
def class_alin(diretorio):
  AX, AY, QX, QY = fiveFoldCrossValidation(0, 9, diretorio, 'png')    #S1
  pS1, aS1 = modeloConvolucional(AX, AY, QX, QY)
  AX, AY, QX, QY = fiveFoldCrossValidation(10, 19, diretorio, 'png')  #S2
  pS2, aS2 = modeloConvolucional(AX, AY, QX, QY)
  AX, AY, QX, QY = fiveFoldCrossValidation(20, 29, diretorio, 'png')  #S3
  pS3, aS3 = modeloConvolucional(AX, AY, QX, QY)
  AX, AY, QX, QY = fiveFoldCrossValidation(30, 39, diretorio, 'png')  #S4
  pS4, aS4 = modeloConvolucional(AX, AY, QX, QY)
  AX, AY, QX, QY = fiveFoldCrossValidation(40, 49, diretorio, 'png')  #S5
  pS5, aS5 = modeloConvolucional(AX, AY, QX, QY)

  print('\n\nResultados class_alin:  5-fold cross validation:\n')
  print('Perda do Teste: ',pS1, pS2, pS3, pS4, pS5, 'Média:', round(np.mean([pS1, pS2, pS3, pS4, pS5]),4))
  print('Acurácia do Teste: ',aS1, aS2, aS3, aS4, aS5, 'Média:', round(np.mean([aS1, aS2, aS3, aS4, aS5]),4))
  print('Erro do Teste: ',round(1-aS1,4), round(1-aS2,4), round(1-aS3,4), round(1-aS4,4), round(1-aS5,4), 'Média:', round(np.mean([1-aS1, 1-aS2, 1-aS3, 1-aS4, 1-aS5]),4))

class_alin('/content/drive/MyDrive/rice_1to50/')

Tamanho das matrizes: AX = (200, 250, 250) AY = (200,) QX = (50, 250, 250) QY = (50,)
Epoch 1/30
2/2 - 2s - loss: 2.5446 - accuracy: 0.1950 - 2s/epoch - 814ms/step
Epoch 2/30
2/2 - 0s - loss: 4.3688 - accuracy: 0.4200 - 220ms/epoch - 110ms/step
Epoch 3/30
2/2 - 0s - loss: 1.6213 - accuracy: 0.6000 - 210ms/epoch - 105ms/step
Epoch 4/30
2/2 - 0s - loss: 0.7001 - accuracy: 0.8300 - 206ms/epoch - 103ms/step
Epoch 5/30
2/2 - 0s - loss: 0.3721 - accuracy: 0.8950 - 208ms/epoch - 104ms/step
Epoch 6/30
2/2 - 0s - loss: 0.2318 - accuracy: 0.9500 - 207ms/epoch - 103ms/step
Epoch 7/30
2/2 - 0s - loss: 0.1367 - accuracy: 0.9800 - 210ms/epoch - 105ms/step
Epoch 8/30
2/2 - 0s - loss: 0.3053 - accuracy: 0.8250 - 206ms/epoch - 103ms/step
Epoch 9/30
2/2 - 0s - loss: 0.1164 - accuracy: 0.9400 - 210ms/epoch - 105ms/step
Epoch 10/30
2/2 - 0s - loss: 0.1228 - accuracy: 0.9450 - 216ms/epoch - 108ms/step
Epoch 11/30
2/2 - 0s - loss: 0.0529 - accuracy: 0.9850 - 208ms/epoch - 104ms/step
Epoch 12/30
2/2 - 0s - l